## Image Augmentation and Transfer Learning for Convolutional Neural Networks

Get the current working directory

In [2]:
import os

PATH = os.getcwd()

Define the data path

In [3]:
# Define data path
data_path = PATH + '/data'
data_dir_list = os.listdir(data_path)

In [4]:
print(data_dir_list)

['cats', 'dogs', 'horses', 'Humans']


In [5]:
!pip install opencv-python

Required variables declaration and initialization

In [6]:
img_rows=224
img_cols=224
num_channel=3
num_epoch=2

img_data_list=[]
classes_names_list=[]

Read the images and store them in the list

In [7]:
import cv2

for dataset in data_dir_list:
    classes_names_list.append(dataset) 
    print ('Loading images from {} folder\n'.format(dataset)) 
    img_list = os.listdir(data_path +'/'+ dataset)
    for img in img_list:
        input_img = cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize = cv2.resize(input_img,(img_rows, img_cols))
        img_data_list.append(input_img_resize)

Loading images from cats folder

Loading images from dogs folder

Loading images from horses folder

Loading images from Humans folder



In [8]:
num_classes = len(classes_names_list)

print(num_classes)

4


###### Image preprocessiong 

In [9]:
import numpy as np
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255

In [10]:
print (img_data)

[[[[0.15686275 0.17254902 0.15294118]
   [0.16470589 0.17254902 0.15686275]
   [0.1764706  0.1764706  0.16078432]
   ...
   [0.7019608  0.8117647  0.81960785]
   [0.654902   0.7882353  0.8       ]
   [0.6313726  0.7764706  0.7882353 ]]

  [[0.16078432 0.1764706  0.15686275]
   [0.16470589 0.17254902 0.15686275]
   [0.1764706  0.1764706  0.16078432]
   ...
   [0.6784314  0.78431374 0.8       ]
   [0.6431373  0.77254903 0.78431374]
   [0.62352943 0.7647059  0.7764706 ]]

  [[0.15686275 0.17254902 0.15294118]
   [0.16078432 0.16862746 0.15294118]
   [0.16862746 0.16470589 0.14901961]
   ...
   [0.654902   0.75686276 0.77254903]
   [0.6431373  0.7647059  0.78039217]
   [0.64705884 0.7764706  0.7921569 ]]

  ...

  [[0.11372549 0.10980392 0.11764706]
   [0.10196079 0.09411765 0.10196079]
   [0.08627451 0.08235294 0.09019608]
   ...
   [0.09803922 0.12156863 0.17254902]
   [0.08235294 0.10588235 0.15686275]
   [0.12941177 0.15294118 0.19607843]]

  [[0.11764706 0.11764706 0.1254902 ]
   [0.1

In [11]:
print (img_data.shape)

img_data = img_data.reshape(img_data.shape[0], img_data.shape[1], img_data.shape[2], num_channel)

(808, 224, 224, 3)


In [12]:
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape

In [13]:
classes = np.ones((num_of_samples,), dtype='int64')
classes[0:202]=0
classes[202:404]=1
classes[404:606]=2
classes[606:]=3

In [14]:
from keras.utils import to_categorical

# convert class labels to on-hot encoding
classes = to_categorical(classes, num_classes)

C:\Users\Sri Yogesh\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Shuffle the dataset

In [15]:
from sklearn.utils import shuffle

x, y = shuffle(img_data, classes, random_state=2)

Split the dataset

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

Defining the model

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [18]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape = input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Compile the model

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

Viewing model_configuration

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 106, 106, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 64)        0         
__________

In [21]:
model.get_config()

[{'class_name': 'Conv2D',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'batch_input_shape': (None, 224, 224, 3),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'data_format': 'channels_last',
   'dilation_rate': (1, 1),
   'dtype': 'float32',
   'filters': 32,
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uniform',
     'mode': 'fan_avg',
     'scale': 1.0,
     'seed': None}},
   'kernel_regularizer': None,
   'kernel_size': (3, 3),
   'name': 'conv2d_1',
   'padding': 'valid',
   'strides': (1, 1),
   'trainable': True,
   'use_bias': True}},
 {'class_name': 'Conv2D',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'data_format': 'channels_last',
   'dilation_rate': (1, 1

In [22]:
model.layers[0].get_config()

{'activation': 'relu',
 'activity_regularizer': None,
 'batch_input_shape': (None, 224, 224, 3),
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'data_format': 'channels_last',
 'dilation_rate': (1, 1),
 'dtype': 'float32',
 'filters': 32,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'kernel_size': (3, 3),
 'name': 'conv2d_1',
 'padding': 'valid',
 'strides': (1, 1),
 'trainable': True,
 'use_bias': True}

In [23]:
model.layers[0].input_shape

(None, 224, 224, 3)

In [24]:
model.layers[0].output_shape

(None, 222, 222, 32)

In [25]:
model.layers[0].get_weights()

[array([[[[-0.0042752 , -0.05344323, -0.05557063, -0.09617606,
            0.04478635,  0.11194691, -0.01040919, -0.1330382 ,
            0.02075943, -0.03168918, -0.00528154, -0.06645468,
           -0.06071475, -0.03249019, -0.04530609, -0.12454037,
           -0.04202313,  0.13434972, -0.11552081,  0.08268821,
            0.01862015,  0.04841165,  0.08527546, -0.12499077,
           -0.10447496,  0.00330125,  0.05608803,  0.03639926,
            0.00545579,  0.01673925, -0.13637945,  0.03444165],
          [-0.10035981, -0.13562356, -0.08364617,  0.08740331,
            0.01397692, -0.05254088,  0.08332327,  0.08718243,
           -0.13441789,  0.06745796,  0.13486253,  0.03477316,
           -0.01809042,  0.04069571, -0.06232709,  0.1191947 ,
            0.08446254, -0.11583212, -0.10403503, -0.10836267,
            0.00602649,  0.02799001,  0.03120071,  0.08582306,
           -0.08634375,  0.01264018, -0.1129126 , -0.12434418,
           -0.05922791, -0.02974994, -0.10497627,  0.1

In [26]:
model.layers[0].trainable

True

#### Training/fit the model 

In [27]:
hist = model.fit(X_train, y_train, batch_size=64, epochs = 25, verbose=1, validation_data=(X_test, y_test))

Train on 646 samples, validate on 162 samples
Epoch 1/25
646/646 [==============================] - 28s 43ms/step - loss: 2.5834 - acc: 0.2601 - val_loss: 1.3856 - val_acc: 0.2654
Epoch 2/25
646/646 [==============================] - 21s 33ms/step - loss: 1.3762 - acc: 0.3142 - val_loss: 1.3751 - val_acc: 0.2901
Epoch 3/25
646/646 [==============================] - 20s 30ms/step - loss: 1.3285 - acc: 0.3777 - val_loss: 1.7093 - val_acc: 0.2099
Epoch 4/25
646/646 [==============================] - 20s 31ms/step - loss: 1.3171 - acc: 0.3638 - val_loss: 2.2533 - val_acc: 0.1975
Epoch 5/25
646/646 [==============================] - 18s 28ms/step - loss: 1.3337 - acc: 0.4087 - val_loss: 1.3851 - val_acc: 0.3025
Epoch 6/25
646/646 [==============================] - 19s 30ms/step - loss: 1.2091 - acc: 0.4598 - val_loss: 1.3530 - val_acc: 0.3642
Epoch 7/25
646/646 [==============================] - 17s 27ms/step - loss: 1.0772 - acc: 0.5526 - val_loss: 1.2637 - val_acc: 0.3951
Epoch 8/25
646/6

#### Training with callbacks

Callback:
    
    A callback is a set of functions to be applied at given stages of the training procedure. 
    
    Callbacks is used to get a view on internal states and statistics of the model during training. 
    
    Pass a list of callbacks to the .fit() method of the Sequential or Model classes. 
        
        The relevant methods of the callbacks will then be called at each stage of the training. 

In [28]:
from keras import callbacks

##### CSVLogger:
    
    Callback that streams epoch results to a csv file.

In [29]:
filename='model_train_new.csv'

csv_log = callbacks.CSVLogger(filename, separator=',', append=False)

##### EarlyStopping:

    Stop training when a monitored quantity has stopped improving.

In [30]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='min')

##### ModelCheckpoint:

    Save the model after every epoch.

In [31]:
filepath="Best-weights-my_model-{epoch:03d}-{loss:.4f}-{acc:.4f}.hdf5"

checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')


##### Create the callbacks list

In [32]:
callbacks_list = [csv_log, early_stopping, checkpoint]

##### Fit the model with the created list of callbacks

In [ ]:
hist = model.fit(X_train, y_train, batch_size=16, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test),callbacks=callbacks_list)

##### Evaluating the model

In [33]:
score = model.evaluate(X_test, y_test, batch_size=16)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

162/162 [==============================] - 4s 22ms/step
Test Loss: 3.7253903926890573
Test Accuracy: 0.5802469135802469


In [34]:
test_image = X_test[0:1]
print (test_image.shape)

(1, 224, 224, 3)


In [35]:
print(model.predict(test_image))
print(model.predict_classes(test_image))
print(y_test[0:1])

[[1.7727200e-06 5.9231446e-04 9.9940586e-01 9.0297297e-11]]
[2]
[[0. 0. 1. 0.]]


##### Printing the confusion matrix

In [36]:
from sklearn.metrics import confusion_matrix

Y_pred = model.predict(X_test)
print(Y_pred)

[[1.77273887e-06 5.92306023e-04 9.99405980e-01 9.02986783e-11]
 [1.16421388e-05 2.76595969e-02 9.72305536e-01 2.31345966e-05]
 [4.32436401e-03 9.70804811e-01 2.05345843e-02 4.33626818e-03]
 [1.00000000e+00 2.82580764e-10 2.37930292e-12 6.69362306e-19]
 [3.73628369e-04 9.98779237e-01 8.47178104e-04 5.15103302e-08]
 [6.57702214e-04 9.80124474e-01 2.25851242e-03 1.69593301e-02]
 [5.65650460e-09 4.62052796e-09 2.64830788e-07 9.99999762e-01]
 [2.46634125e-04 2.37663556e-02 9.75986958e-01 9.83502058e-10]
 [2.59913140e-15 1.00000000e+00 1.56781923e-12 1.78664914e-14]
 [2.52286688e-15 5.14538578e-10 2.63650300e-12 1.00000000e+00]
 [5.93437254e-01 1.11829257e-03 4.04851735e-01 5.92656957e-04]
 [4.02998414e-19 1.27981637e-13 1.67223987e-15 1.00000000e+00]
 [2.80994253e-04 5.43091670e-02 7.25309998e-02 8.72878790e-01]
 [1.23459401e-07 2.83801910e-02 2.36625256e-05 9.71595943e-01]
 [5.33964310e-04 3.73665243e-01 6.25695407e-01 1.05373889e-04]
 [1.01330901e-15 5.19895821e-07 9.99999523e-01 7.265864

In [37]:
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)

[2 2 1 0 1 1 3 2 1 3 0 3 3 3 2 2 0 1 3 1 1 3 1 1 0 1 1 3 0 3 2 2 2 2 0 3 3
 1 3 2 2 3 1 3 0 1 1 0 2 1 2 0 1 1 2 2 0 2 1 0 3 1 3 3 3 2 3 2 0 0 1 3 1 0
 3 1 2 3 0 3 1 2 3 2 2 2 3 0 1 2 2 1 1 2 3 1 2 2 2 3 0 1 1 3 3 3 0 2 3 2 0
 3 2 1 0 0 3 2 1 0 1 2 3 1 2 2 1 1 3 0 2 3 2 1 3 3 2 2 3 0 1 0 3 3 3 0 1 1
 3 1 0 2 1 3 3 3 3 2 1 2 1 0]


In [38]:
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

[[18 15  3  5]
 [ 2 18 11  9]
 [ 3  3 26  1]
 [ 5  8  3 32]]


###### Saving and loading model and weights

In [39]:
from keras.models import model_from_json, load_model

In [40]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")

In [41]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")

In [42]:
model.save('model.hdf5')
loaded_model=load_model('model.hdf5')

###### Transfer Learning - 1

![](img/vggnet_architecture.png)

In [43]:
from keras.layers.core import Dense
from keras.layers import Input

In [44]:
# Custom_vgg_model_1
#Training the classifier alone
image_input = Input(shape=(224, 224, 3))

In [45]:
from keras.applications.vgg16 import VGG16

vgg_model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')


include_top: whether to include the 3 fully-connected layers at the top of the network.

weights: one of None (random initialization) or 'imagenet' (pre-training on ImageNet).

input_tensor: optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model.

classes: optional number of classes to classify images into, only to be specified if include_top is True, and if no weights argument is specified.


In [46]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [49]:
last_layer = vgg_model.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)

In [50]:
from keras.models import Model

custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [51]:
for layer in custom_vgg_model.layers[:-3]:
    layer.trainable = False

In [52]:
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [55]:
custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
hist = custom_vgg_model.fit(X_train, y_train, batch_size=32, epochs=30, verbose=1, validation_data=(X_test, y_test))

Train on 646 samples, validate on 162 samples
Epoch 1/30
646/646 [==============================] - 36s 55ms/step - loss: 11.9014 - acc: 0.2616 - val_loss: 12.8348 - val_acc: 0.2037
Epoch 2/30
608/646 [===========================>..] - ETA: 2s - loss: 11.7439 - acc: 0.2714

In [ ]:
(loss, accuracy) = custom_vgg_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))

###### Transfer Learning - 2

In [ ]:
# Training the feature extraction also
model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')

model.summary()

In [ ]:
last_layer = model.get_layer('block5_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_vgg_model2 = Model(image_input, out)

In [ ]:
custom_vgg_model2.summary()

In [ ]:
# freeze all the layers except the dense layers
for layer in custom_vgg_model2.layers[:-3]:
    layer.trainable = False

In [ ]:
custom_vgg_model2.summary()

In [ ]:
custom_vgg_model2.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

In [ ]:
hist = custom_vgg_model2.fit(X_train, y_train, batch_size=32, epochs=8, verbose=1, validation_data=(X_test, y_test))

In [ ]:
(loss, accuracy) = custom_vgg_model2.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

Ref:
    
    https://github.com/anujshah1003/Transfer-Learning-in-keras---custom-data